# EdgeFactory : testing computation of edge co-occurences

This notebook regroups different visual tests to check for the computation of edge co-occurences.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
#matplotlib.rcParams.update({'font.size': 18, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
matplotlib.rcParams.update({'text.usetex': False})
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(precision=4)#, suppress=True)
# http://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
#%load_ext autoreload
#%autoreload 2

# where should we store the figures generated by this notebook
figpath = ''

In [3]:
%cd -q ../probe

## defining framework

In [4]:
from SparseEdges import SparseEdges
mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
mp.pe.N = 4
mp.pe.do_mask = False
mp.pe.MP_alpha = 1.
mp.pe.do_whitening = False
mp.init()

print ('Range of spatial frequencies: ', mp.sf_0)

IndentationError: unexpected indent (SparseEdges.py, line 473)

In [ ]:
help(SparseEdges.cooccurence)

In [ ]:
help(SparseEdges.cooccurence_hist)

In [ ]:
print ('Range of angles: ', mp.theta*180./np.pi)

NOTE: everything below this point should go to another notebook

## Computing edge statistics 

In [ ]:
from SparseEdges import SparseEdges

In [ ]:
mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
mp.pe.figsize_edges = 12
print (mp.pe)

In [ ]:
mp.pe.n_levels = np.log2(mp.pe.N_X)

### Basics

#### setting up coordinates conventions

cardinal points in X, Y (clockwise):  E (=right=3 o'clock=horizontal), ESE, S, WSW, W, WNW, N, ENE 

In [ ]:
X = np.array([0, 1./4, 1./2, 1./4, 0, -1./4, -1./2])#, -1./4])\n",
Y = np.array([1./2, np.sqrt(3)/4, 0, -np.sqrt(3)/4, -1./2, -np.sqrt(3)/4, 0.])#, np.sqrt(3)/
Theta = np.arctan2(-X, Y)
Theta = ((Theta + np.pi/2  - np.pi/mp.pe.N_phi/2 ) % (np.pi)) - np.pi/2  + np.pi/mp.pe.N_phi/2
print (Theta* 180./np.pi)

The X, Y image axis are defined in the matrix style (origin on the upper left; X going vertically down, Y going right horizontally) - the angle is defined as the angle wrt to the horizontal axis :

In [ ]:
fig, a = mp.show_edges(np.vstack(((X/2. + .5)*mp.pe.N_X, (Y/2. + .5)*mp.pe.N_X, Theta, .01*np.ones(X.shape), np.ones(X.shape), np.zeros(X.shape) )))
t = [a.text(Y_, X_, np.str((X_, Y_, angle*180/np.pi))) for (X_, Y_, angle)  in zip((X/2. + .5)*mp.pe.N_X, (Y/2. + .5)*mp.pe.N_X, Theta)]

#### Testing some angle calculations

In [ ]:
# test points
X = np.array([0.25, 0.25, 0.75])
Y = np.array([0.25, 0.75, 0.25])
X, Y = X*mp.pe.N_X, Y*mp.pe.N_X
print ('coord = ', X, Y)
# by definition, theta = 0 is the horizontal
Theta = np.array([0., np.pi/4, np.pi/2])
print ('angle (CCW wrt horizontal) =', Theta * 180./np.pi)
Sf_0, Weights, Phases = .01*np.ones(X.shape), np.ones(X.shape), np.ones(X.shape)
fig, a = mp.show_edges(np.vstack((X, Y, Theta, Sf_0, Weights, Phases)))
#t = [a.text(Y_, X_, np.str((X_, Y_, '%0.3f ' % (angle*180/np.pi), i))) for (X_, Y_, angle, i)  in zip((X/2. + .5)*N_X, (Y/2. + .5)*N_Y, Theta, range(3))]
t = [a.text(Y_, X_, np.str(i)) for (X_, Y_, i)  in zip(X, Y, range(3))]

#### Relative distance

In [ ]:
dx = X[:, np.newaxis] - X[np.newaxis, :]
dy = Y[:, np.newaxis] - Y[np.newaxis, :]
print ('dx_ij = \n', dx, '\n dy_ij = \n', dy)

In [ ]:
d = np.sqrt(dx**2 + dy**2) #/ pe.N_X * pe.d_width # distance in visual angle
print( np.diag(d), d[0, :])
plt.matshow(d); plt.colorbar()

In [ ]:
theta = Theta[:, np.newaxis] - Theta[np.newaxis, :]
print ('angle to self = ', np.diag(theta), '\nangle from 2 to ...= ', theta[2, :]*180/np.pi)
plt.matshow(theta*180/np.pi); plt.colorbar(ticks=np.linspace(-90, 90, 5))
print (theta*180/np.pi)

In [ ]:
theta = ((theta + np.pi/2 - np.pi/mp.pe.N_Dtheta/2)  % (np.pi) ) - np.pi/2  + np.pi/mp.pe.N_Dtheta/2
plt.matshow(theta*180/np.pi); plt.colorbar(ticks=np.linspace(-90, 90, 5))
print (np.diag(theta).std(), theta.min()*180/np.pi, theta.max()*180/np.pi)
print (theta*180/np.pi)

In [ ]:
# still anti-symmetric (theta == - theta.T)
plt.matshow(np.sin(theta + theta.T)); plt.colorbar()

phi is the azimuth of the other edge wrt a reference, the matrix gives phi_ij = phi of j wrt i

In [ ]:
print ((np.arctan2(dy, dx) - np.pi/2)*180/np.pi, Theta*180/np.pi)
#np.seterr(all='ignore')
phi = (np.arctan2(dy, dx) - np.pi/2 - Theta[np.newaxis, :] + np.pi ) % (2*np.pi)  - np.pi
phi += np.diag(np.nan*np.ones(3))
#print np.isnan(phi)
#phi[np.isnan(phi)] = 0.
print( phi*180/np.pi)# np.diag(phi, 1)*180/np.pi, phi[0, :]*180/np.pi
plt.matshow(phi*180/np.pi); plt.colorbar(ticks=np.linspace(-90, 90, 5))

In [ ]:
# not symmetric
plt.matshow(np.sin(phi - phi.T)); plt.colorbar()

In [ ]:
theta = Theta[:, np.newaxis] - Theta[np.newaxis, :]
phi = np.arctan2(dy, dx) - np.pi/2 - Theta[np.newaxis, :]
phi += np.diag(np.nan*np.ones(3))
psi = phi - theta/2
#psi = (Theta[:, np.newaxis]+Theta[np.newaxis, :])/2 + np.arctan2(-dx, dy)# + np.pi/2
print( psi*180/np.pi)#, (psi - (psi + psi.T)/2)*180/np.pi
plt.matshow(psi*180/np.pi); plt.colorbar()

In [ ]:
# symmetric
plt.matshow(np.sin(psi - psi.T)); plt.colorbar()

if we assume that an angle occurrence may also happen wrt to its mirror symetric (flipping left and right), then $\psi$ becomes non-oriented

In [ ]:
# putting everything in the right range:
psi = ((psi + np.pi/2  - np.pi/mp.pe.N_phi/2 ) % (np.pi)) - np.pi/2  + np.pi/mp.pe.N_phi/2
print (psi*180/np.pi)
plt.matshow(psi*180/np.pi); plt.colorbar()
#  still symmetric
plt.matshow(psi - psi.T); plt.colorbar()
#print np.sin(psi - psi.T).std()

In [ ]:
level = np.log2(Sf_0[:, np.newaxis]) - np.log2(Sf_0[np.newaxis, :])
plt.matshow(level); plt.colorbar()

In [ ]:
weights = Weights[:, np.newaxis] * Weights[np.newaxis, :]
plt.matshow(weights); plt.colorbar()
weights /= (d + 1.e-6)
plt.matshow(weights); plt.colorbar()

In [ ]:
weights[d==0.] = 0. # exclude self-occurence
plt.matshow(weights); plt.colorbar()

### testing the plotting functions using synthetic edge images

In [ ]:
help( SparseEdges.histedges_theta)
help( SparseEdges.histedges_scale)
help( SparseEdges.cohistedges)

In [ ]:
def myplot(edgeslist, mp):
    """
    a function to display edges

    """

    fig = plt.figure(figsize=(9, 12))
    a1 = fig.add_subplot(321)
    fig, a = mp.show_edges(edgeslist[:, :, 0], image=np.ones((mp.pe.N_X, mp.pe.N_X)), fig=fig, ax=a1)
    a2 = fig.add_subplot(322)
    fig, a2 = mp.histedges_scale(edgeslist, fig=fig, ax=a2)
    a3 = fig.add_subplot(323, polar=True)
    fig, a3 = mp.histedges_theta(edgeslist, fig=fig, ax=a3)
    a4 = fig.add_subplot(324)
    v_hist = mp.cohistedges(edgeslist, display='none')
    v_hist_nosym = mp.cohistedges(edgeslist, display='none', symmetry=False)
    fig, a4 = mp.cohistedges(edgeslist=None, v_hist=v_hist, fig=fig, ax=a4)
    a5 = fig.add_subplot(325)
    fig, a5 = mp.cohistedges(edgeslist=None, v_hist=v_hist_nosym, fig=fig, ax=a5, symmetry=False, display='colin_geisler')
    a6 = fig.add_subplot(326)
    fig, a6 = mp.cohistedges(edgeslist=None, v_hist=v_hist_nosym, fig=fig, ax=a6, symmetry=False, display='cocir_geisler')
    #plt.tight_layout()
    plt.show()
    fig2 = plt.figure(figsize=(9, 9))
    fig2, (a1, a2, a3, a4) = mp.cohistedges(edgeslist=None, v_hist=v_hist, fig=fig2, display='full')
    return v_hist

#### Just with 2 edges and  averaged over N_images trials

In [ ]:
mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
mp.pe.N_image = 20
mp.pe.d_max = 2.
mp.pe.d_min = 0.
edgeslist = np.zeros((6, 2, mp.pe.N_image))
angle = np.pi/6
for i_N, sign in enumerate([-1., 1.]):
    edgeslist[0, i_N, :] = mp.pe.N_X *(1/2. + .1 * np.sin(angle) * sign) + .0005 * np.random.randn(mp.pe.N_image)
    edgeslist[1, i_N, :] = mp.pe.N_X *(1/2. - .1 * np.cos(angle) * sign) + .0005 * np.random.randn(mp.pe.N_image)
    edgeslist[2, i_N, :] = (angle + np.pi/360.* np.random.randn(mp.pe.N_image)) % np.pi
    edgeslist[3, i_N, :] = .03
    edgeslist[4, i_N, :] = 1.
#print  edgeslist
v_hist = myplot(edgeslist, mp=mp)
#print v_hist

In [ ]:
edgeslist = np.zeros((6, 2, mp.pe.N_image))
angle = np.pi/6
for i_N, sign in enumerate([-1., 1.]):
    edgeslist[0, i_N, :] = mp.pe.N_X *(1/2. + .1 * np.sin(angle) * sign) + .0005 * np.random.randn(mp.pe.N_image)
    edgeslist[1, i_N, :] = mp.pe.N_X *(1/2. + .1 * np.cos(angle) * sign) + .0005 * np.random.randn(mp.pe.N_image)
    edgeslist[2, i_N, :] = (angle + np.pi/360.* np.random.randn(mp.pe.N_image)) % np.pi
    edgeslist[3, i_N, :] = .03
    edgeslist[4, i_N, :] = 1.
#print  edgeslist
_ = myplot(edgeslist, mp=mp)

In [ ]:
edgeslist = np.zeros((6, 2, mp.pe.N_image))
angle = np.pi/6
for i_N, sign in enumerate([-1., 1.]):
    edgeslist[0, i_N, :] = mp.pe.N_X *(1/2. + .1 * np.sin(angle) * sign) + .0005 * np.random.randn(mp.pe.N_image)
    edgeslist[1, i_N, :] = mp.pe.N_X *(1/2. + .1 * np.cos(angle) * sign) + .0005 * np.random.randn(mp.pe.N_image)
    edgeslist[2, i_N, :] = sign*(angle + np.pi/360.* np.random.randn(mp.pe.N_image)) % np.pi
    edgeslist[3, i_N, :] = .03
    edgeslist[4, i_N, :] = 1.
_ = myplot(edgeslist, mp=mp)

#### random edges

In [ ]:
mp.pe.N = 512
print (mp.pe.base_levels, mp.n_levels)
edgeslist = np.zeros((6, mp.pe.N, mp.pe.N_image))
edgeslist[0, :, :] = mp.pe.N_X * np.random.rand(mp.pe.N, mp.pe.N_image)
edgeslist[1, :, :] = mp.pe.N_X * np.random.rand(mp.pe.N, mp.pe.N_image)
edgeslist[2, :, :] = ((np.pi* np.random.rand(mp.pe.N, mp.pe.N_image) ) % np.pi) - np.pi/2
edgeslist[3, :, :] =  .03*np.ceil(mp.pe.base_levels**(mp.n_levels*(np.random.rand(mp.pe.N, mp.pe.N_image)+.5)))
edgeslist[4, :, :] = np.random.rand(mp.pe.N, mp.pe.N_image) 
edgeslist[5, :, :] = 2*np.pi*np.random.rand(mp.pe.N, mp.pe.N_image)

_ = myplot(edgeslist, mp=mp)

#### parallel edges

In [ ]:
edgeslist = np.zeros((6, mp.pe.N, mp.pe.N_image))
for i_N, x in enumerate(np.linspace(-mp.pe.N_X/4., mp.pe.N_X/4., mp.pe.N)):
    edgeslist[0, i_N, :] = mp.pe.N_X * np.random.rand(mp.pe.N_image)
    edgeslist[1, i_N, :] = mp.pe.N_X * np.random.rand(mp.pe.N_image)
    edgeslist[2, i_N, :] = (np.pi/6 + np.pi/360.* np.random.randn(mp.pe.N_image) ) % np.pi
    edgeslist[3, i_N, :] = mp.pe.base_levels**np.ceil(mp.n_levels*np.random.rand(mp.pe.N_image))
    edgeslist[4, i_N, :] = np.random.randn(mp.pe.N_image)**2
#! show one sample
_ = myplot(edgeslist, mp=mp)

#print mp.cohistedges(edgeslist, display='none')

#### colinear edges

In [ ]:
edgeslist = np.zeros((6, mp.pe.N, mp.pe.N_image))
for i_N, x in enumerate(np.linspace(-mp.pe.N_X/4., mp.pe.N_X/4., mp.pe.N)):
    edgeslist[0, i_N, :] = mp.pe.N_X/2. + x + 1. * np.random.randn(mp.pe.N_image)
    edgeslist[1, i_N, :] = mp.pe.N_X/2. + x + 1. * np.random.randn(mp.pe.N_image)
    edgeslist[2, i_N, :] = ((-np.pi/4 + np.pi*5./180.* np.random.randn(mp.pe.N_image) ) % np.pi) - np.pi/2
    edgeslist[3, i_N, :] = 32. #np.sqrt(2.)/2 * N_X / N
    edgeslist[4, i_N, :] = 1.
_ = myplot(edgeslist, mp=mp)#

#### cocircular edges

In [ ]:
#! cocircular edges:
edgeslist = np.zeros((6, mp.pe.N, mp.pe.N_image))
#for i_N, angle in enumerate(np.linspace(0, 2*np.pi, N)):
for i_N, angle in enumerate(2*np.pi*np.random.rand(mp.pe.N)):
    edgeslist[0, i_N, :] = mp.pe.N_X/2. - mp.pe.N_X/4.*np.sin(angle) + .5 * np.random.randn(mp.pe.N_image)
    edgeslist[1, i_N, :] = mp.pe.N_X/2. + mp.pe.N_X/4.*np.cos(angle) + .5 * np.random.randn(mp.pe.N_image)
    edgeslist[2, i_N, :] = ((np.pi + angle + .5*np.pi/180 * np.random.randn(mp.pe.N_image)) % np.pi) - np.pi/2
    edgeslist[3, i_N, :] = 0.03
    edgeslist[4, i_N, :] = 1.
_ = myplot(edgeslist, mp=mp)#

#### cocircular edges on a wider radius

In [ ]:
edgeslist = np.zeros((6, mp.pe.N, mp.pe.N_image))
#for i_N, angle in enumerate(np.linspace(0, 2*np.pi, pe.N)):
for i_N, angle in enumerate(2*np.pi*np.random.rand(mp.pe.N)):
    edgeslist[0, i_N, :] = mp.pe.N_X/2. - .95*mp.pe.N_X/2.*np.sin(angle) + .05 * np.random.randn(mp.pe.N_image)
    edgeslist[1, i_N, :] = mp.pe.N_X/2. + .95*mp.pe.N_X/2.*np.cos(angle) + .05 * np.random.randn(mp.pe.N_image)
    edgeslist[2, i_N, :] = ((np.pi + angle + .5*np.pi/180 * np.random.randn(mp.pe.N_image)) % np.pi) - np.pi/2
    edgeslist[3, i_N, :] = 0.03
    edgeslist[4, i_N, :] = 1.
_ = myplot(edgeslist, mp=mp)

### real-life examples

In [ ]:
mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
help( mp.run_mp)

In [ ]:
def full_hist(edgeslist, **kwargs):
        v_hist = np.zeros((mp.pe.N_r, mp.pe.N_phi, mp.pe.N_Dtheta, mp.pe.N_scale, edgeslist.shape[2])) # bins=(self.edges_d, self.edges_phi, self.edges_theta, self.edges_loglevel)
        for i_image in range(edgeslist.shape[2]):
            v_hist[..., i_image] = mp.cohistedges(edgeslist[:, :, i_image][..., np.newaxis], display='none', **kwargs)
            v_hist[..., i_image] /= v_hist[..., i_image].sum()
        return v_hist

In [ ]:
#! using first database
#print im.get_imagelist(exp='testing_vanilla', name_database='serre07_distractors')
#!ls mat/edges/testing_vanilla_serre07_distractors/

#### plotting with distractors

In [ ]:
#edgeslist = np.load('../AssoField/mat/classifier_serre07_distractors_edges.npy')
mp.pe.datapath = '../../SLIP/database/'
imageslist, edgeslist, RMSE = mp.process(exp='Sparselets', name_database='serre07_distractors')

In [ ]:
#_ = myplot(edgeslist[:, :, 0][:, :, np.newaxis], pe=pe)

In [ ]:
import os
matname = os.path.join(mp.pe.matpath, 'Sparselets_serre07_distractors_hist.npy')
try:
    v_hist = np.load(matname)
except Exception as e:
    import os
    if not(os.path.isfile(matname + '_lock')):
        print( 'There is no histogram, creating one: ', e)
        open(matname + '_lock', 'w').close() # touching
        v_hist = myplot(edgeslist, mp=mp)
        np.save(matname, v_hist)
        os.remove(matname + '_lock')
    else:
        print (' Some process is building the histogram ' + matname   ) 

#### the same with more noise

In [ ]:
imageslist, edgeslist, RMSE = mp.process(exp='testing_noise', name_database='serre07_distractors')

matname = os.path.join(mp.pe.matpath, 'testing_noise_vanilla_serre07_distractors_hist.npy')
try:
    v_hist = np.load(matname)
except Exception as e:
    if not(os.path.isfile(matname + '_lock')):
        print ('There is no histogram, creating one: ', e)
        open(matname + '_lock', 'w').close() # touching
        v_hist = myplot(edgeslist, mp=mp)
        np.save(matname, v_hist)
        os.remove(matname + '_lock')
    else:
        print (' Some process is building the histogram ' + matname)

       
#edgeslist = np.load('../AssoField/mat/classifier_noise_serre07_targets_edges.npy')
#! show one sample
#fig, a = mp.show_edges(edgeslist[:, :, 0], image=None)
#v = pylab.axis((0, pe.N_X, pe.N_X, 0))

#### plotting with targets

using second database

In [ ]:
%ls ../probe/cache_dir/*_edges.npy

In [ ]:
#! using second database
#!----------------------
#imageslist, edgeslist, RMSE = mp.process(exp='testing_vanilla', name_database='serre07_targets')
edgeslist = np.load('cache_dir/testing_vanilla_serre07_targets_edges.npy')
_ = myplot(edgeslist[:, :, 0][:, :, np.newaxis], pe=pe)

In [ ]:
v_hist = mp.cohistedges(edgeslist, display='none')

In [ ]:
#!rm mat/testing_vanilla_serre07_targets_hist.npy*
matname = os.path.join(mp.pe.matpath, 'testing_vanilla_serre07_targets_hist.npy')
try:
    v_hist = np.load(matname)
except Exception as e:
    if not(os.path.isfile(matname + '_lock')):
        print ('There is no histogram, creating one: ', e)
        open(matname + '_lock', 'w').close() # touching
        v_hist = myplot(edgeslist, mp=mp)
        np.save(matname, v_hist)
        os.remove(matname + '_lock')
    else:
        print (' Some process is building the histogram ' + matname)


Note that the histogram is symmetric with respect to log-scales

In [ ]:
mp.init_binedges()
print (mp.binedges_d.max())
print (v_hist.shape)
print (mp.binedges_loglevel)

In [ ]:
l_ = v_hist.sum(axis=(2, 1, 0))
print (l_, l_-l_[::-1])

In [ ]:
print (np.sum(v_hist), np.sum((v_hist - v_hist[:, :, :, ::-1])**2))

In [ ]:
v_hist[:, 4, 4, 1]*100, v_hist[:, 4, 4, 3]*100

In [ ]:
np.set_printoptions(precision=5, suppress=True)
v_hist_chevrons = v_hist.sum(axis=(0,3))
print (v_hist_chevrons.shape)

In [ ]:
print (v_hist_chevrons)
print (v_hist_chevrons[:, 6:11])
print (v_hist_chevrons[:, :6:-1])


In [ ]:
_ = myplot(edgeslist, mp=mp)

## some book keeping for the notebook

In [ ]:
%load_ext watermark
%watermark

In [ ]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, sympy

In [ ]:
%cd -q ../notebooks